In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [2]:
#Data parameters
image_size = 28*28
n_class = 10

#Train parameters
learning_rate = 0.1
total_steps = 1000
batch_size = 100

X = tf.placeholder(tf.float32, [None, image_size], name='InputImage')
Y = tf.placeholder(tf.float32, [None, n_class], name='Label')

In [ ]:
def conv_layer(self, filters):
    self = tf.nn.conv2d(self, filters=filters, activation='relu')
    return self

def max_pool_layer(self, shape):
    self = tf.nn.max_pool(self, ksize=shape, strides=1)
    return self
    
def build_cnn():
    input_layer = tf.reshape(X, [-1, 28, 28, 1])
    with tf.name_scope('CNN'):
        cnn = conv_layer(input_layer, 64)
        cnn = max_pool_layer(cnn)
        cnn = conv_layer(cnn, 32)
        cnn = max_pool_layer(cnn)
    with tf.name_scope('DNN')
        
    